In [237]:
import requests
#http://www.newyorksocialdiary.com/party-pictures?page=1
url = "http://www.newyorksocialdiary.com/party-pictures"
#response = requests.get(url, params={"limit": 1000, "offset": 0})
response = requests.get(url,params={'page':3})
print response.url
print response.text[:200] + "..."

http://www.newyorksocialdiary.com/party-pictures?page=3
<!DOCTYPE html>
  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->
  <!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->
  <!...


In [240]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
print soup.prettify()

<!DOCTYPE html>
<!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->
<!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->
<!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]-->
<!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><![endif]-->
<!--[if !IE]><!-->
<html class="no-js" dir="ltr" lang="en" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#">
 <!--<![endif]-->
 <head>
  <title>
   

In [297]:
import pandas as pd
def scrap1(soup):
    parent_div = soup.select('div[class=view-content]')[0] #Find (at most) *one*
    a=parent_div.select('span[class=field-content]')
    #print a
    N=len(a)
    event=[a[i].text for i in range(N) if (i % 2)==0]
    date=[a[i].text for i in range(N) if (i % 2)==1]
    link=[a[i].select('a')[0]['href'] for i in range (N) if (i%2)==0]
    df=pd.DataFrame({'event':event,'date':date,'link':link})
    return (df)


df=pd.DataFrame(columns=['event','date','link'])
for page in range (27):
    response = requests.get(url,params={'page':page})
    soup = BeautifulSoup(response.text)
    ans=scrap1(soup)
    df=pd.concat([df,ans])

#Updating index
df.index=range(len(df.index))

    


,date,event,link
0,"Tuesday, March 15, 2016","Breakfasts, dinners, and fetes",/party-pictures/2016/breakfasts-dinners-and-fetes
1,"Sunday, March 13, 2016",Preserving and protecting,/party-pictures/2016/preserving-and-protecting
2,"Friday, March 4, 2016","Bunny Hop, Purim Ball, and POSH",/party-pictures/2016/bunny-hop-purim-ball-and-...
3,"Wednesday, March 2, 2016","NYBG Orchid Dinner, NYPL Family Benefit, and V...",/party-pictures/2016/nybg-orchid-dinner-nypl-f...
4,"Friday, February 26, 2016",A good mix of guests,/party-pictures/2016/a-good-mix-of-guests
5,"Tuesday, February 23, 2016",Hearts and Crosses,/party-pictures/2016/hearts-and-crosses
6,"Thursday, February 18, 2016",Celebratory evenings,/party-pictures/2016/celebratory-evenings
7,"Wednesday, February 10, 2016",Preservation and care,/party-pictures/2016/preservation-and-care
8,"Friday, February 5, 2016",The Young Professionals,/party-pictures/2016/the-young-professionals
9,"Monday, February 1, 2016",Heroes in History,/party-pictures/2016/heroes-in-history


In [296]:
len(df.index)

1316

In [284]:
df.

,date,event,link
0,"Monday, March 10, 2014",Faces and Places,/party-pictures/2014/faces-and-places
1,"Friday, March 7, 2014",Hopping Along ...,/party-pictures/2014/hopping-along
2,"Thursday, March 6, 2014",New York City Delights,/party-pictures/2014/new-york-city-delights
3,"Wednesday, March 5, 2014",Keeping warm down in Palm Beach,/party-pictures/2014/keeping-warm-down-in-palm...
4,"Tuesday, March 4, 2014",2014 VANITY FAIR Oscar Party,/party-pictures/2014/2014-vanity-fair-oscar-party
5,"Friday, February 28, 2014",Enchanted Evenings,/party-pictures/2014/enchanted-evenings
6,"Wednesday, February 26, 2014",Winter Social Schedule,/party-pictures/2014/winter-social-schedule
7,"Monday, February 24, 2014",The Winter Ball,/party-pictures/2014/the-winter-ball
8,"Thursday, February 20, 2014",Dining with the Divas,/party-pictures/2014/dining-with-the-divas
9,"Thursday, February 13, 2014",Staying Warm,/party-pictures/2014/staying-warm


In [260]:
soup

<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]--><!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]--><!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]--><!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]--><!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><![endif]--><!--[if !IE]><!--><html class="no-js" dir="ltr" lang="en" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><!--<![endif]-->\n<head>\n<title>Party Pictur

In [236]:
from datetime import datetime
datetime.strptime('Tuesday, March 15, 2016','%A, %B %d, %Y')
type(pd.Series(df.date)[1])

unicode

In [210]:
a[0].select('a')[0]['href']

'/party-pictures/2016/breakfasts-dinners-and-fetes'

In [28]:
print soup.select('div#view-content')

[]
